# Tarea 1 Análisis de Datos

In [1]:
%matplotlib inline
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import dates as mpdates
import datetime
import pandas as pd
import geopandas as gpd


#

In [ ]:

df1 = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto74/paso_a_paso_std.csv', 
                     encoding='utf-8',
                    sep=',')

df2=gpd.read_file('https://opendata.arcgis.com/datasets/1c64fcb18f5a41e088b25ef9f42b58d7_0.geojson') # contiene población de cada comuna

In [ ]:
pd.set_option("display.max_columns", None)  #se quitan límites del display de dataframes
pd.set_option("display.max_rows", None)

# DF1

In [ ]:
#se limpia y moldea el df1 por conveniencia

df1['Fecha']=pd.to_datetime(df1['Fecha'])
df1=df1.rename(columns={'codigo_region': 'cod_region','region_residencia':'region','codigo_comuna':'cod_comuna',
                      'comuna_residencia':'comuna','Fecha':'fecha'})
df1=df1.loc[df1['Paso'] == 1]
df1['zona']=df1['zona'].str.replace('Urbana','u')
df1['zona']=df1['zona'].str.replace('Rural','r')
df1['zona']=df1['zona'].str.replace('Total','t')    
print('total de datos:',len(df1))
df1.info()
df1.head(10)

In [ ]:
df1['zona'].value_counts()

In [ ]:
df1_otra_zona=df1.loc[(df1['zona'] != 'u')] 
df1_otra_zona=df1_otra_zona.loc[df1['zona'] != 'r']
df1_otra_zona=df1_otra_zona.loc[df1['zona'] != 't']
# df1_otra_zona.head(300)       
#                               Se decidió NO considerar las zonas que no estaban especificadas como urbanas, rurales o totales ya que generarian errores
#                               en los números finales.

In [ ]:
df1_r=df1.loc[df1['zona'] == 'r']
df1_u=df1.loc[df1['zona'] == 'u']
df1_t=df1.loc[df1['zona'] == 't']
# se divide el df en zonas por conveniencia a futuro

# DF2

In [ ]:
df2.head(1)

In [ ]:
# se limpia el df2, se dejó la columna "geometry" para hacer un choropleth pero no resultó
df2=df2[['COMUNA','T_POB_R','T_POB_U','T_POB','geometry']]
df2=df2.rename(columns={'COMUNA':'cod_comuna','T_POB_R':'población_rural','T_POB_U':'población_urbana','T_POB':'población_total'})
df2['cod_comuna']=pd.to_numeric(df2['cod_comuna'], errors='coerce')
df2.info()
df2.head(2)


In [ ]:
df2.loc[df2['cod_comuna']==2101].head(10)

In [ ]:
# df2.loc[:,['cod_comuna','población_rural']]
# df2.loc[:,['cod_comuna','población_urbana']]
# df2.loc[:,['cod_comuna','población_total']]
df1_r=pd.merge(df1_r,df2.loc[:,['cod_comuna','población_rural']],how='left',on='cod_comuna')
df1_u=pd.merge(df1_u,df2.loc[:,['cod_comuna','población_urbana']],how='left',on='cod_comuna')
df1_t=pd.merge(df1_t,df2.loc[:,['cod_comuna','población_total']],how='left',on='cod_comuna')
df1_r=df1_r.rename(columns={'población_rural':'población'})
df1_u=df1_u.rename(columns={'población_urbana':'población'})
df1_t=df1_t.rename(columns={'población_total':'población'})

### dataframes que se separaron en zonas se les agrega una población acorde a comuna y zona

In [ ]:
df=pd.merge(df1_r,df1_u,how='outer')   #merge rural con urbano
print(df['población'].isnull().sum())

In [ ]:
df=pd.merge(df,df1_t,how='outer')      #merge (rural y urbano) con 
df=df.sort_values(['fecha','cod_comuna'])
print(df['población'].isnull().sum())
df.info()

In [ ]:
print(df['fecha'].min(),'  ',df['fecha'].max())
df.head(2)

## Separar por regiones y buscar día con mayor número de personas en cuarentena

In [ ]:
df_region=df[['cod_region','region']]    #df con cod_region y region asociada
df_region=df_region.drop_duplicates()


In [ ]:
def region_plot(x,df,dfreg):    ####### función para crear los plots de cada región individual
    data_reg=df.loc[df['cod_region']==x]


    data = data_reg.groupby(['fecha'], as_index= False)['población'].sum() # https://www.codegrepper.com/code-examples/python/how+to+sum+the+revenue+from+every+day+in+a+dataframe+python
    # data['población'].max()
    plt.rcParams["figure.figsize"] = [9, 6]
    plt.rcParams["figure.autolayout"] = True
    
    plt.plot(data['fecha'], data['población'],linewidth=3 )
    
    n='Región de '+dfreg.loc[dfreg['cod_region']==x]['region'].values[0]+' (max = '+data['población'].max().astype(str)+')'  #label del titulo
    plt.title(n, fontsize=25)
    plt.ylabel('Población en cuarentena',fontsize=20)
    plt.xlabel('Fecha ',fontsize=20)
    plt.xticks(rotation=45)
    plt.show()
    
    print('Dias con máximo de personas en cuarentena:')
    print(data.loc[data['población']==data['población'].max()]['fecha'].head())

# Cantidad diaria de personas en cuarentena por región.
#### Cada gráfico establece su punto máximo de personas en el título


In [ ]:
for i in range(1,17):
    region_plot(i,df,df_region)

# A nivel país.

In [ ]:
def pais_plot(df):
    #data_reg=df.loc[df['cod_region']==x]


    data = df.groupby(['fecha'], as_index= False)['población'].sum() # https://www.codegrepper.com/code-examples/python/how+to+sum+the+revenue+from+every+day+in+a+dataframe+python
   
    plt.rcParams["figure.figsize"] = [12, 8]
    plt.rcParams["figure.autolayout"] = True
    
    plt.plot(data['fecha'], data['población'],linewidth=3 )
    
    n='Personas en cuarentena por día'+' (max = '+data['población'].max().astype(str)+')'
    plt.title(n, fontsize=30)
    plt.ylabel('Población en cuarentena',fontsize=25)
    plt.xlabel('Fecha ',fontsize=25)
    plt.xticks(rotation=45)
    plt.show()
    print('Dias con máximo de personas en cuarentena:')
    print(data.loc[data['población']==data['población'].max()]['fecha'].head())

In [ ]:
pais_plot(df)

# Todas las regiones.

In [ ]:
def all_regs_plot(df,dfreg):
    

    plt.rcParams["figure.figsize"] = [15, 9]
    plt.rcParams["figure.autolayout"] = True
    
    for i in range(1,17):
        
        data_reg=df.loc[df['cod_region']==i]
        data = data_reg.groupby(['fecha'], as_index= False)['población'].sum() # https://www.codegrepper.com/code-examples/python/how+to+sum+the+revenue+from+every+day+in+a+dataframe+python
        plt.plot(data['fecha'], data['población'],linewidth=2 ,label=dfreg.loc[dfreg['cod_region']==i]['region'].values[0])
    
    #n='Región de '+dfreg.loc[dfreg['cod_region']==x]['region'].values[0]+' (max = '+data['población'].max().astype(str)+')'
    plt.title('Personas en cuarentena por región', fontsize=25)
    plt.ylabel('Población en cuarentena',fontsize=20)
    plt.xlabel('Fecha ',fontsize=20)
    plt.legend(loc='upper right')
    plt.xticks(rotation=45)
    plt.show()
    

In [ ]:
all_regs_plot(df,df_region)